In [ ]:
import os, time, subprocess, pydicom
import torchio as tio
from torchio import ScalarImage

# 找到包含最多 DICOM 文件的子文件夹
def find_largest_dicom_folder(base_path):
    dicom_folder = None
    max_dicom_count = 0

    for root, dirs, files in os.walk(base_path):
        dicom_count = 0
        for file in files:
            try:
                pydicom.dcmread(os.path.join(root, file), stop_before_pixels=True)
                dicom_count += 1
            except:
                continue

        if dicom_count > max_dicom_count:
            max_dicom_count = dicom_count
            dicom_folder = root

    print(f"📁 Selected DICOM folder: {dicom_folder} (with {max_dicom_count} DICOM files)")
    return dicom_folder

# 主处理逻辑
for patient_id in range(7000, 7050):  # 可以改成多个病人范围
    Label = 'LF'
    base_path = f'/nfs/turbo/coe-mreedsensitive/_PatientScans/Runoff_2024/{Label}{patient_id}'

    if not os.path.exists(base_path):
        print(f"⚠️ Base path not found: {base_path}")
        continue

    print(f"\n🚀 Processing patient LF{patient_id}")
    
    
    # 更新后的保存路径
    resampled_dir = '/home/cuixing/Foot&Ankle/Data_Predict/Predict_Data'
    os.makedirs(resampled_dir, exist_ok=True)
    resampled_path = os.path.join(resampled_dir, f'{Label}{patient_id}_resampled_0000.nii.gz')

    if not os.path.exists(resampled_path):
        StartTime = time.time()

        dicom_path = find_largest_dicom_folder(base_path)
        if dicom_path is None:
            print(f"❌ No valid DICOM folder found for patient {Label}{patient_id}")
            continue

        image = ScalarImage(dicom_path)
        new_resolution = (0.75, 0.75, 0.75)
        resample = tio.Resample(new_resolution)
        resampled_image = resample(image)
        resampled_image.save(resampled_path)
        print(f"✅ Resampled image saved to {resampled_path}")

        EndTime = time.time()
        print(f"⏱ Resampling time: {EndTime - StartTime:.2f} seconds")
    else:
        print(f"📄 Using existing resampled file for patient {Label}{patient_id}")

#     # TotalSegmentator 分割
#     StartTime = time.time()
#     output_path = os.path.join(base_path, f'LF{patient_id}_Segmentations')
#     os.makedirs(output_path, exist_ok=True)

#     command = ["TotalSegmentator", "-i", resampled_path, "-o", output_path]

#     try:
#         result = subprocess.run(command, check=True, capture_output=True, text=True)
#         print(result.stdout)
#         print(result.stderr)
#     except subprocess.CalledProcessError as e:
#         print(f"❌ Error during segmentation for patient LF{patient_id}: {e}")
#         print(e.stdout)
#         print(e.stderr)
#         continue

#     EndTime = time.time()
#     print(f"✅ Segmentation completed in {EndTime - StartTime:.2f} seconds")
#     print(f"📂 Output saved to: {output_path}")


⚠️ Base path not found: /nfs/turbo/coe-mreedsensitive/_PatientScans/Runoff_2024/LF7000

🚀 Processing patient LF7001
📄 Using existing resampled file for patient LF7001

🚀 Processing patient LF7002
📄 Using existing resampled file for patient LF7002

🚀 Processing patient LF7003
📁 Selected DICOM folder: /nfs/turbo/coe-mreedsensitive/_PatientScans/Runoff_2024/LF7003/LF7003_CT AORTA RUNOFF ARTERIES ABD WO AND W CONTRAST/AP RUNOFF CTA 2.5mm_190550 (with 1008 DICOM files)
✅ Resampled image saved to /home/cuixing/Foot&Ankle/Data_Predict/Predict_Data/LF7003_resampled_0000.nii.gz
⏱ Resampling time: 97.50 seconds

🚀 Processing patient LF7004
📁 Selected DICOM folder: /nfs/turbo/coe-mreedsensitive/_PatientScans/Runoff_2024/LF7004/LF7004_CT AORTA RUNOFF ARTERIES ABD WO AND W CONTRAST/CAP RUNOFF CTA 2.5mm_130350 (with 1142 DICOM files)
✅ Resampled image saved to /home/cuixing/Foot&Ankle/Data_Predict/Predict_Data/LF7004_resampled_0000.nii.gz
⏱ Resampling time: 96.71 seconds

🚀 Processing patient LF7005

In [33]:
import nibabel as nib
import numpy as np

# 指定路径
nifti_path = "/home/cuixing/Foot&Ankle/Data_Predict/Predict_Data/LF2009_resampled_0000.nii.gz"

# 加载图像
img = nib.load(nifti_path)
data = img.get_fdata()
affine = img.affine

# 修改 affine 的 z 轴 spacing 为正值
affine[2, 2] = abs(affine[2, 2])

# 保存新的图像
fixed_img = nib.Nifti1Image(data, affine, img.header)
nib.save(fixed_img, nifti_path)

# 打印结果
print("Affine matrix:\n", affine[2, 2])


Affine matrix:
 [[-7.50000000e-01  0.00000000e+00 -0.00000000e+00  2.00015625e+02]
 [ 0.00000000e+00 -7.50000000e-01 -0.00000000e+00  1.92115631e+02]
 [ 0.00000000e+00  0.00000000e+00  7.50000000e-01 -1.04663000e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
